In [ ]:
import sys
sys.path.append('../')

import numpy as np
import scipy
import tqdm
import utils
import geometry
from phantom import Phantom
from tissue import Tissue
from transducer import Transducer, Focused, Planewave
from transducer_set import TransducerSet
from simulation import SimProperties, Simulation
from sensor import Sensor
from experiment import Experiment
from reconstruction import Reconstruction, DAS, Compounding 

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import kwave
import glob

phantom = Phantom(source_path = None,
            voxel_dims = (0.1e-3,0.1e-3,0.1e-3), # size of each voxel
            matrix_dims = (400,400,400), # number of voxels along each axis
            baseline = (1500, 1000), # background medium speed of sound and density
            seed = 5678,)

kidney_sos = # speed of sound in kidney m/s
kidney_rho = # density of kidney kg/m^3
kidney_sigma = # standard deviation for gaussian noise
kidney_noise_scale = # spatial period for noise in m
fat = Tissue(name='fat', c=1450, rho=920, sigma=10, scale=0.0001, label=1)
kidney = Tissue(name='kidney', c=kidney_sos, rho=kidney_rho, sigma=kidney_sigma, 
                scale=kidney_noise_scale, label=2) 
#...

pos = 0.7

phantom.add_tissue_sphere((pos,-pos,0), 8, fat) # sphere center, radius, tissue makeup
#...

plt.imshow(phantom.get_complete()[0,:,phantom.matrix_dims[2]//4, :], 
           vmin=np.amin(phantom.get_complete()[0]), vmax=np.amax(phantom.get_complete()[0]))
plt.colorbar()

n_transducers = # number of transducers in trajectory

transducers = [Planewave(max_frequency = # [Hz],
                         elements= 32,
                         element_width = 1e-4, # [m]
                         elevation = 1e-4 * 32, 
                         sweep = np.pi/3, # angles from -60 to 60 
                         ray_num = 64, # number of steering angles in the range prescribed by sweep 
                         focus_elevation = # wherever you'd like to put the elevation focus
                         )
                for i in range(n_transducers)]

for t in transducers:
    t.make_sensor_coords(phantom.baseline[0])

transducer_set = TransducerSet(transducers, seed=8888)

# global coordinate system origin (0,0,0) at centroid of phantom as defined above
for i in range(n_transducers):
    transducer_transform = Transform(translation=(0,0,0), rotation=(0,0,0)) # global coords of transducer centroid, zyx intrinsic tait-bryan
    transducer_set.assign_pose(i, )

transducer_set.plot_transducer_fovs(scale=0.05)
transducer_set.plot_transducers()

sensor = Sensor(transducer_set=transducer_set)

simprops = SimProperties(grid_size   = (1024,128,128), # you may need to tweak the grid size for maximum efficency
                 PML_size    = (32,8,8),
                 PML_alpha   = 2,
                 t_end       = 12e-5,           # [s]
                 bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                 alpha_coeff = 0.5, 	        # [dB/(MHz^y cm)]
                 alpha_power = 1.5,
                 )

test_experiment = Experiment(
                 simulation_path = '', # your desired path
                 sim_properties  = simprops,
                 phantom         = phantom,
                 transducer_set  = transducer_set,
                 sensor          = sensor,
                 nodes           = 2, # how many compute nodes you plan to use
                 results         = None,
                 indices         = None,
                 workers         = 3, # how many workers/node
                 )

test_experiment.save()

test_experiment.run()

test_experiment.add_results()

reconstruction = Compounding(test_experiment) # beta, may not currently work as intended



